# 🏍️ LossZero: Motorcycle Night Ride SegFormer-B2 Optimized

이 노트북은 **SegFormer-B2** 모델을 사용하여 야간 오토바이 주행 이미지의 시멘틱 세그멘테이션을 수행합니다.

### 🛠️ 주요 시나리오
- **모델**: SegFormer-B2 (Transformer 기반)
- **백본**: MiT-B2
- **사전 학습**: Cityscapes (도로 환경 특화)
- **최적화**: AdamW + FP16 Mixed Precision
- **손실 함수**: Weighted CrossEntropy (중요 객체 가중치 부여)

In [1]:
# wandb 설치
!pip install wandb

In [4]:
import os
import cv2
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from pycocotools.coco import COCO
import albumentations as A
from albumentations.pytorch import ToTensorV2
from transformers import SegformerForSemanticSegmentation, SegformerConfig
from torch.amp import autocast, GradScaler
from tqdm.auto import tqdm
import wandb
wandb.login()

print(f"PyTorch version: {torch.__version__}")


wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Create a new API key at: https://wandb.ai/authorize?ref=models
wandb: Store your API key securely and do not share it.
wandb: Paste your API key and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vlux22 (vlux22-free) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


PyTorch version: 2.9.0+cpu


## Colab 연결

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
def get_device():
    if torch.cuda.is_available():
        return "cuda"
    # SegFormer의 permute 연산과 MPS 백엔드 간에 충돌
    # elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available() and torch.backends.mps.is_built():
    #     return "mps"
    return "cpu"

# ⚙️ 설정 (Configuration)
# DATA_DIR = os.path.expanduser("~/Projects/LossZero/data/Motorcycle Night Ride Dataset")
DATA_DIR = os.path.expanduser("/content/drive/MyDrive/motor_model")
JSON_PATH = os.path.join(DATA_DIR, "COCO_motorcycle (pixel).json")
IMG_DIR = os.path.join(DATA_DIR, "images")

CFG = {
    "project": "LossZero",
    "model_name": "nvidia/segformer-b2-finetuned-cityscapes-1024-1024",
    "img_size": (352, 352),
    "batch_size": 4,
    "lr": 1e-4,
    "epochs": 25,
    "device": get_device()
}

print(f"Using device: {CFG['device']}")

Using device: cpu


In [7]:
def create_mask_from_json(coco, img_id, img_info, id_to_idx):
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)

    for ann in anns:
        cat_id = ann['category_id']
        if cat_id in id_to_idx:
            cls_idx = id_to_idx[cat_id]
            pixel_mask = coco.annToMask(ann)
            mask[pixel_mask == 1] = cls_idx

    return mask

def process_single_data(coco, img_id, img_dir, id_to_idx, transform=None):
    img_info = coco.loadImgs(img_id)[0]
    img_path = os.path.join(img_dir, img_info['file_name'])

    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    mask = create_mask_from_json(coco, img_id, img_info, id_to_idx)

    if transform:
        augmented = transform(image=image, mask=mask)
        image, mask = augmented['image'], augmented['mask']

    return image, torch.as_tensor(mask).long()

train_transform = A.Compose([
    A.Resize(CFG['img_size'][0], CFG['img_size'][1]),

    # --- 야간 전용 Augmentation 추가 ---
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.RandomGamma(gamma_limit=(80, 120), p=0.5), # 어두운 저조도 개선
    A.GaussNoise(std_range=(0.02, 0.05), p=0.3), # 야간 노이즈 대응

    # --- 기하학적 변형 (데이터 수 보충용) ---
    A.HorizontalFlip(p=0.5), # 좌우 반전
    # 0.0625는 머신러닝/딥러닝 커뮤니티에서 오랫동안 검증된 '사실상 표준(De Facto Standard)
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5), # 이동/크기/회전

    # ImageNet 데이타셋의 평균값 나쁘지 않음. SegFormer가 ImageNet/Cityscapes로 배웠으니까
    # 모델이 새로운 사진을 받을 때: 입력_이미지 = (원본_이미지 - 평균) / 표준편차
    # 이렇게 계산해주면, 어떤 사진이 들어와도 "평균이 0이고 표준편차가 1인(Standard Normal Distribution)" 아주 예쁜 데이터로 변신
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

coco = COCO(JSON_PATH)
img_ids = list(coco.imgs.keys())
cat_ids = coco.getCatIds()
id_to_idx = {cat_id: i for i, cat_id in enumerate(cat_ids)}
print(f"Category Mapping: {id_to_idx}")

/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


loading annotations into memory...
Done (t=12.88s)
creating index...
index created!
Category Mapping: {1329681: 0, 1323885: 1, 1323884: 2, 1323882: 3, 1323881: 4, 1323880: 5}


## Traing / Val 분리

In [8]:
class MotorcycleNightRideDataset(Dataset):
    def __init__(self, coco_annotation_path, img_dir, id_to_idx, transform=None):
        # 각 데이터셋(train/val)마다 자체 COCO 객체를 로드
        self.coco = COCO(coco_annotation_path)
        self.img_ids = list(self.coco.imgs.keys())
        self.img_dir = img_dir
        self.id_to_idx = id_to_idx
        self.transform = transform

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        # process_single_data 함수는 이미 정의되어 있습니다.
        image, mask = process_single_data(self.coco, img_id, self.img_dir, self.id_to_idx, self.transform)
        return image, mask

# 검증 데이터셋을 위한 변환 (Augmentation 제외)
val_transform = A.Compose([
    A.Resize(CFG['img_size'][0], CFG['img_size'][1]),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# 훈련 및 검증 데이터셋 경로
train_annotation_path = os.path.join(DATA_DIR, "annotations", "instances_train.json")
val_annotation_path = os.path.join(DATA_DIR, "annotations", "instances_val.json")
train_img_dir = os.path.join(IMG_DIR, "train") # DATA_DIR/images/train 경로 사용
val_img_dir = os.path.join(IMG_DIR, "val")     # DATA_DIR/images/val 경로 사용

# 데이터셋 인스턴스 생성
train_dataset = MotorcycleNightRideDataset(
    coco_annotation_path=train_annotation_path,
    img_dir=train_img_dir,
    id_to_idx=id_to_idx,
    transform=train_transform
)

val_dataset = MotorcycleNightRideDataset(
    coco_annotation_path=val_annotation_path,
    img_dir=val_img_dir,
    id_to_idx=id_to_idx,
    transform=val_transform
)

# 데이터 로더 생성
train_loader = DataLoader(
    train_dataset,
    batch_size=CFG['batch_size'],
    shuffle=True,
    num_workers=os.cpu_count(),
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CFG['batch_size'],
    shuffle=False, # 검증 데이터는 셔플할 필요 없음
    num_workers=os.cpu_count(),
    pin_memory=True
)

print(f"훈련 데이터셋 크기: {len(train_dataset)}개 이미지")
print(f"검증 데이터셋 크기: {len(val_dataset)}개 이미지")
print(f"훈련 데이터로더 배치 수: {len(train_loader)}개")
print(f"검증 데이터로더 배치 수: {len(val_loader)}개")

loading annotations into memory...
Done (t=4.32s)
creating index...
index created!
loading annotations into memory...
Done (t=0.77s)
creating index...
index created!
훈련 데이터셋 크기: 160개 이미지
검증 데이터셋 크기: 40개 이미지
훈련 데이터로더 배치 수: 40개
검증 데이터로더 배치 수: 10개


### 📉 클래스별 분포 요약 (내림차순)

1. **Undrivable (주행 불가 영역)**: **42.9%** (압도적 1위)
   - 배경(하늘, 건물, 풀숲 등)이 이미지의 절반 가까이 차지합니다.
2. **Road (주행 가능 도로)**: **27.1%**
   - 도로 자체도 꽤 많은 영역을 차지합니다.
3. **My bike (내 오토바이)**: **15.8%**
   - 주행자 시점이라 내 오토바이가 항상 보이기 때문에 비율이 높습니다.
4. **Rider (탑승자)**: **8.1%**
   - 다른 오토바이 운전자나 내 신체가 포함된 것으로 보입니다.
5. **Moveable (이동 물체)**: **4.7%**
   - 다른 차량, 보행자 등 안전에 가장 중요한 장애물인데 비율이 매우 낮습니다.
6. **Lane Mark (차선)**: **1.4%**
   - 가장 심각한 불균형입니다. 도로 주행의 핵심인 차선이 고작 1% 남짓입니다.

In [9]:
id2label = {i: coco.loadCats(cat_id)[0]['name'] for cat_id, i in id_to_idx.items()}
label2id = {v: k for k, v in id2label.items()}

model = SegformerForSemanticSegmentation.from_pretrained(
    CFG['model_name'],
    num_labels=len(id_to_idx),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
).to(CFG['device'])

optimizer = torch.optim.AdamW(model.parameters(), lr=CFG['lr'], weight_decay=0.01)

# ⚖️ 클래스별 가중치 설정 (Class Weights)
weights = torch.tensor([
    5.0,   # Rider (8.1%) -> 적당히 높임
    2.0,   # My bike (15.8%) -> 낮춤 (이미 많음)
    10.0,  # Moveable (4.7%) -> 강력하게 높임
    20.0,  # Lane Mark (1.4%) -> 아주 강력하게!! (핵심)
    1.0,   # Road (27.1%) -> 기본
    0.5    # Undrivable (42.9%) -> 낮춤 (너무 많아서 방해됨)
], dtype=torch.float).to(CFG['device'])

criterion = nn.CrossEntropyLoss(weight=weights)

scaler = GradScaler('cuda') if CFG['device'] == 'cuda' else None

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/380 [00:00<?, ?it/s]

SegformerForSemanticSegmentation LOAD REPORT from: nvidia/segformer-b2-finetuned-cityscapes-1024-1024
Key                           | Status   |                                                                                                  
------------------------------+----------+--------------------------------------------------------------------------------------------------
decode_head.classifier.bias   | MISMATCH | Reinit due to size mismatch ckpt: torch.Size([19]) vs model:torch.Size([6])                      
decode_head.classifier.weight | MISMATCH | Reinit due to size mismatch ckpt: torch.Size([19, 768, 1, 1]) vs model:torch.Size([6, 768, 1, 1])

Notes:
- MISMATCH	:ckpt weights were loaded, but they did not match the original empty weight shapes.


In [ ]:
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 1. 데이터 분할은 MotorcycleNightRideDataset에서 이미 처리되었으므로, 여기서는 DataLoader를 사용합니다.
# 기존의 train_test_split 및 수동 데이터 로딩 로직을 제거합니다.

print("🚀 SegFormer-B2 Training Start...")

wandb.init(project=CFG['project'], config=CFG)

for epoch in range(CFG['epochs']):
    # --- Training Phase ---
    model.train()
    train_loss_sum = 0

    # Train Loop: train_loader 사용
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]")
    for images, masks in pbar:
        X = images.to(CFG['device']).contiguous()
        y = masks.to(CFG['device']).contiguous()

        optimizer.zero_grad()

        if scaler:
            with torch.amp.autocast('cuda'):
                outputs = model(X).logits.contiguous()
                upsampled_logits = nn.functional.interpolate(
                    outputs,
                    size=y.shape[-2:],
                    mode="bilinear",
                    align_corners=False
                ).contiguous()
                loss = criterion(upsampled_logits, y)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(X).logits.contiguous()
            upsampled_logits = nn.functional.interpolate(
                outputs,
                size=y.shape[-2:],
                mode="bilinear",
                align_corners=False
            ).contiguous()
            loss = criterion(upsampled_logits, y)
            loss.backward()
            optimizer.step()

        train_loss_sum += loss.item()
        pbar.set_postfix(Train_Loss=f"{loss.item():.4f}")

    avg_train_loss = train_loss_sum / len(train_loader)

    # --- Validation Phase ---
    model.eval()
    val_loss_sum = 0

    with torch.no_grad():
        # Val Loop: val_loader 사용
        for images, masks in val_loader:
            X = images.to(CFG['device']).contiguous()
            y = masks.to(CFG['device']).contiguous()

            outputs = model(X).logits.contiguous()
            upsampled_logits = nn.functional.interpolate(
                outputs,
                size=y.shape[-2:],
                mode="bilinear",
                align_corners=False
            ).contiguous()

            loss = criterion(upsampled_logits, y)
            val_loss_sum += loss.item()

    avg_val_loss = val_loss_sum / len(val_loader)

    wandb.log({'Train Loss': avg_train_loss, 'Val Loss': avg_val_loss, 'epoch': epoch})

    print(f"📝 Epoch [{epoch+1}/{CFG['epochs']}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    # 모델 저장 (Best Val Loss일 때 저장하는 로직 추가 가능)
    # torch.save(model.state_dict(), f"segformer_epoch_{epoch+1}.pth")

wandb.finish()

🚀 SegFormer-B2 Training Start...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 1 [Train]:   0%|          | 0/40 [00:00<?, ?it/s]

📝 Epoch [1/25] Train Loss: 1.2706 | Val Loss: 0.8211


Epoch 2 [Train]:   0%|          | 0/40 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7eec3cf7d4e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^Exception ignored in: ^^<function _MultiProcessingDataLoaderIter.__del__ at 0x7eec3cf7d4e0>
^Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()^
^  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
^^    if w.is_alive():
 ^     ^ ^^^^^^^^^^^^^^

📝 Epoch [2/25] Train Loss: 0.6854 | Val Loss: 0.6047


Epoch 3 [Train]:   0%|          | 0/40 [00:00<?, ?it/s]

📝 Epoch [3/25] Train Loss: 0.5126 | Val Loss: 0.5644


Epoch 4 [Train]:   0%|          | 0/40 [00:00<?, ?it/s]